# Detecting bias in crime data 

## part 1: Getting to know the data

Margriet Groenendijk -  28 September 2020 \
**Data Science Lunch & Learn on [Crowdcast](https://www.crowdcast.io/e/data-science-lunch-and)**

This is part 1 of a series about detecting bias in data. Various methods will be explored including different fairness metrics and comparing bias changes both over time and space. 

In this first notebook the data will be explored to understand what variables are available and which ones to use for a further analysis. There will be lots of data processing and data visualisations to get a better feel for what we can do in coming sessions to detect bias. 


* Run this Jupyter notebook in **Watson Studio**
    * sign up for a free [IBM Cloud account](http://ibm.biz/crowdcast_289) 
    * Follow [these instructions](https://github.com/IBMDeveloperUK/data-science-lunch-and-learn/blob/master/watson-studio-instructions.md) 
* Or in any other environment


## Contents

0. [Installing and loading packages]()
1. [Loading the data]() \
    1.1 [Using the API]() \
    1.2 [Downloading all data as csv files]()
2. [Street level crimes]()    
3. [Stop and searches]()
4. [Outcomes]()
5. [Join datasets]()
6. [Maps]()
7. [Summary]()
8. [Open questions and ideas]()


## 0. Installing and loading packages

### Run the notebook in Watson Studio

TODO: 
* add data to Cloud Object Storage
* use SQL Query to access the data from notebook


### Run the notebook locally

Open a terminal and go to the folder with the notebook. Then create and activate a new environment:

`conda env create --name crime-uk python=3.6` \
`conda activate crime-uk` 

Install the packages needed:

`conda install jupyter requests dask pandas geopandas ` \
`conda install matplotlib seaborn ` 

Add the new environment to a kernel:

`python -m ipykernel install --user --name crime-uk --display-name "Python3.6 (crime-uk)"` 

Save environment to yaml file for reproducibility:

`conda env export --name crime-uk > crime-uk.yml`

Start the notebook environment:

`jupyter notebook`

Uncomment the lines below for any packages that are still missing in your environment.

In [ ]:
import sys
# !conda install --yes --prefix {sys.prefix} requests
# !conda install --yes --prefix {sys.prefix} dask
# !conda install --yes --prefix {sys.prefix} geopandas
# !conda install --yes --prefix {sys.prefix} matplotlib
# !conda install --yes --prefix {sys.prefix} seaborn
# !conda install --yes --prefix {sys.prefix} -c conda-forge descartes

In [ ]:
import requests
import glob
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import geopandas as gpd
import dask.dataframe as dd

## 1. Loading the data

### 1.1 Using the API

[A list of all available data sets](https://data.police.uk/docs/method/crimes-street-dates/)

In [ ]:
data_list = requests.get('https://data.police.uk/api/crimes-street-dates')  
print(data_list.status_code)

In [ ]:
data_list_json = data_list.json() 
data_list_df = pd.json_normalize(data_list_json)
data_list_df.head()

In [ ]:
# months with data
data_months = data_list_df['date'].unique()
print(np.sort(data_months))

In [ ]:
# force IDs
force_IDs = data_list_df['stop-and-search'][0]
print(force_IDs)

In [ ]:
# list of forces
forces_list_df = pd.json_normalize(requests.get('https://data.police.uk/api/forces').json())
forces_list_df.head()

### 1.2 Downloading all data as csv files

The rest of the API does not seem to work, so downloaded all [latest data](https://data.police.uk/data/archive/latest.zip) from [here](https://data.police.uk/about/) (21GB!).
 
`force_IDs = ['avon-and-somerset', 'bedfordshire', 'btp', 'cambridgeshire', 'cheshire', 'city-of-london', 'cleveland', 'cumbria', 'derbyshire', 'dorset', 'durham', 'dyfed-powys', 'essex', 'gloucestershire', 'gwent', 'hampshire', 'hertfordshire', 'humberside', 'kent', 'lancashire', 'leicestershire', 'merseyside', 'metropolitan', 'norfolk', 'northamptonshire', 'northumbria', 'north-wales', 'north-yorkshire', 'south-wales', 'south-yorkshire', 'staffordshire', 'suffolk', 'surrey', 'thames-valley', 'warwickshire', 'west-mercia', 'west-midlands', 'west-yorkshire']`

In [ ]:
!wget -N https://data.police.uk/data/archive/latest.zip

In [ ]:
# TODO: when new file available (once a month): unzip and move to data/

This is a large dataset that is organized in folders and files by month and force. With [dask](https://docs.dask.org/en/latest/) you can explore all data without having to load it all into memory. 

There are three different data files. Load data for only one force into three Dask dataframes:

In [ ]:
#force = 'city-of-london'
force = 'metropolitan'
#force = 'hampshire'

# get lists of all files for one force
street_filenames = glob.glob("/Users/work/Data/UK-crime-data/*/*-"+force+"-street.csv")
outcomes_filenames = glob.glob("/Users/work/Data/UK-crime-data/*/*-"+force+"-outcomes.csv")
stop_and_search_filenames = glob.glob("/Users/work/Data/UK-crime-data/*/*-"+force+"-stop-and-search.csv")

# Dask dataframes are not loaded in memory and can load multiple files
street_dd = dd.read_csv("/Users/work/Data/UK-crime-data/*/*-"+force+"-street.csv",dtype={'Crime ID': 'object',
       'Last outcome category': 'object'})
outcomes_dd = dd.read_csv("/Users/work/Data/UK-crime-data/*/*-"+force+"-outcomes.csv")
stop_and_search_dd = dd.read_csv("/Users/work/Data/UK-crime-data/*/*-"+force+"-stop-and-search.csv", 
                                 dtype={'Object of search': 'object'})

print("Number of files:")
print(len(street_filenames))
print(len(outcomes_filenames))
print(len(stop_and_search_filenames))

## 2. Street level crimes

[Data description](https://data.police.uk/docs/method/crime-street/)

* [pandas DataFrame API](https://pandas.pydata.org/pandas-docs/stable/reference/frame.html)
* [Reshaping and pivot tables](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html)

In [ ]:
# Load data into pandas dataframes
street_df = street_dd.compute()
print(len(street_df))

In [ ]:
# drop duplicates - check if these really are duplicates!
street_df2 = street_df.drop_duplicates()
print(len(street_df2))
street_df.head().transpose()

In [ ]:
# drop columns with same value for all rows
print(street_df['Reported by'].unique())
print(street_df['Falls within'].unique())
print(street_df['Context'].unique())

street_df = street_df.drop(columns=['Reported by', 'Falls within', 'Context'])

In [ ]:
# convert Month to datetime
street_df['Month'] = pd.to_datetime(street_df['Month'], format='%Y-%m')
street_df.head()

In [ ]:
print(street_df['Crime type'].unique())

In [ ]:
print(street_df['Last outcome category'].unique())

In [ ]:
# highest number of crimes by type
bytype = street_df.groupby(['Last outcome category','Crime type']).count()
bytype = bytype.drop(columns=['Month','LSOA code','Longitude','Latitude','LSOA name','Location'])
bytype = bytype.rename(index=str, columns={"Crime ID": "Number of crimes"})
bytype.sort_values(by=['Number of crimes'], ascending=False).head(10)

In [ ]:
# highest number of crimes by location
bystreet = street_df.groupby(['Location','Crime type']).count()
bystreet = bystreet.drop(columns=['Month', 'Last outcome category','LSOA code'])
bystreet = bystreet.rename(index=str, columns={"Crime ID": "Number of crimes"})
bystreet.sort_values(by=['Number of crimes'], ascending=False).head()

In [ ]:
# highest number of crimes by area
byarea = street_df.groupby(['LSOA name','Crime type']).count()
byarea = byarea.drop(columns=['Month', 'Location','Longitude','Latitude','Last outcome category','LSOA code'])
byarea = byarea.rename(index=str, columns={"Crime ID": "Number of crimes"})
byarea.sort_values(by=['Number of crimes'], ascending=False).head(10)

In [ ]:
fig = plt.figure();
street_df['Crime type'].groupby(street_df['Crime type']).count().plot.barh(figsize=(14,8));
plt.ylabel(None);
plt.title(force);

In [ ]:
fig = plt.figure();
street_df['Crime type'].groupby(street_df['Month']).count().plot(figsize=(14,6));
plt.title(force);

In [ ]:
# group by crime type
street_type = street_df.groupby(['Month','Crime type'])['Location'].count().unstack(fill_value=0)
street_type.head()

In [ ]:
fig = plt.figure();
street_type.plot(figsize=(14,6));
plt.title(force);

In [ ]:
fig = plt.figure();
street_type.plot(figsize=(14,6));
plt.ylabel('arrests / month', fontsize=16);
plt.xlabel(None);
plt.title(force);
plt.legend(bbox_to_anchor=(1.02, 1.0));

In [ ]:
# group by outcome
street_outcome = street_df.groupby(['Month','Last outcome category'])['Location'].count().unstack(fill_value=0)
street_outcome.head()

In [ ]:
fig = plt.figure();
street_outcome.plot(figsize=(14,6));
plt.ylabel('outcomes / month', fontsize=16);
plt.xlabel(None);
plt.title(force);
plt.legend(bbox_to_anchor=(1.02, 1.0));

In [ ]:
fig = plt.figure();
street_outcome.plot(figsize=(14,6),logy=True);
plt.ylabel('outcomes / month', fontsize=16);
plt.xlabel(None);
plt.title(force);
plt.legend(bbox_to_anchor=(1.02, 1.0));

> **Question: is data updated when there are updates later in time?** For instance when going from `Awaiting court outcome' to `Offender given...`

In [ ]:
fig = plt.figure();
ax=street_type.hist(figsize=(14,14));
ax[0][0].set_xlim((0, 200))

# 3. Stop and searches

In [ ]:
stop_and_search_df = stop_and_search_dd.compute()
print(len(stop_and_search_df))

In [ ]:
# drop duplicates - check if these really are duplicates!
stop_and_search_df = stop_and_search_df.drop_duplicates()
print(len(stop_and_search_df))
stop_and_search_df.head().transpose()

In [ ]:
# drop columns with same value for all rows
print(stop_and_search_df['Policing operation'].unique())
print(stop_and_search_df['Part of a policing operation'].unique())

stop_and_search_df = stop_and_search_df.drop(columns=['Policing operation', 'Part of a policing operation'])

In [ ]:
# convert Month to datetime
stop_and_search_df['Date'] = pd.to_datetime(stop_and_search_df['Date'], format='%Y-%m-%dT%H:%M:%S')
stop_and_search_df['Year'] = stop_and_search_df['Date'].dt.year
stop_and_search_df['Month'] = pd.to_datetime(stop_and_search_df['Date']).dt.to_period('M')
stop_and_search_df.head()

In [ ]:
stop_and_search_grouped1 = stop_and_search_df.groupby(['Gender','Age range'])['Date'].count().unstack(fill_value=0)
stop_and_search_grouped1.head()

In [ ]:
stop_and_search_grouped2 = stop_and_search_df.groupby(['Gender','Object of search'])['Date'].count().unstack(fill_value=0)
stop_and_search_grouped2.head().transpose()

In [ ]:
stop_and_search_grouped3 = stop_and_search_df.groupby(['Gender','Self-defined ethnicity'])['Date'].count().unstack(fill_value=0)
stop_and_search_grouped3.head().transpose()

## 4. Outcomes

In [ ]:
outcomes_df = outcomes_dd.compute()
print(len(outcomes_df))

In [ ]:
# drop duplicates - check if these really are duplicates!
#outcomes_df = outcomes_df.drop_duplicates()
#print(len(outcomes_df))
#outcomes_df.head().transpose()

In [ ]:
# drop columns with same value for all rows
print(outcomes_df['Reported by'].unique())
print(outcomes_df['Falls within'].unique())

outcomes_df = outcomes_df.drop(columns=['Reported by', 'Falls within'])

In [ ]:
# convert Month to datetime
outcomes_df['Month'] = pd.to_datetime(outcomes_df['Month'], format='%Y-%m')
outcomes_df.head()

In [ ]:
outcomes_df['Outcome type'].unique()

## 5. Join datasets

First explore if it is possible to combine the datasets.

Are there duplicates in the data?

[`duplicated`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html)

In [ ]:
#duplicates = outcomes_df.drop_duplicates()
#duplicates.head()

In [ ]:
#print('all data: '+ str(len(outcomes_df)))
#print('no duplicates: '+ str(len(duplicates)))
#print('nr of unique IDs: '+ str(len(outcomes_df_duplicates['Crime ID'].unique())))
#outcomes_df_duplicates.head()

In [ ]:
#outcomes_df = outcomes_df.drop_duplicates()

In [ ]:
# 1, 10, 100, 1000, 1005

i = -2

id = outcomes_df['Crime ID'].iloc[i]
lat = outcomes_df['Latitude'].iloc[i]
lon = outcomes_df['Longitude'].iloc[i]
mon = outcomes_df['Month'].iloc[i]
print(id)
print(lat)
print(lon)
print(mon)

outcomes_df[outcomes_df['Crime ID'] ==id]

In [ ]:
street_df[street_df['Crime ID'] ==id]

In [ ]:
stop_and_search_df[(stop_and_search_df['Latitude']==lat) & (stop_and_search_df['Longitude']==lon)]

In [ ]:
stop_and_search_df[(stop_and_search_df['Latitude']==lat) & (stop_and_search_df['Longitude']==lon) 
                          & (stop_and_search_df['Month']==mon)]

### join street_df and outcomes_df



In [ ]:
crimes_df = pd.merge(street_df, outcomes_df, how='outer', sort=True,on='Crime ID')
crimes_df.head().transpose()

## 6. Maps

kml files for all regions from [here](https://data.police.uk/data/boundaries/).

### Lond boroughs
Explore with this geo-json file for London:

In [ ]:
# load data from a url
london_boroughs = gpd.read_file("https://skgrange.github.io/www/data/london_boroughs.json")

In [ ]:
london_boroughs.plot();

In [ ]:
london_boroughs.plot(column='code');

In [ ]:
[fig, ax] = plt.subplots(1, figsize=(10, 6))
london_boroughs.plot(ax=ax);
ax.axis('off');

In [ ]:
london_boroughs.head()

In [ ]:
from shapely.geometry import Point, LineString, Polygon

# convert to geopandas dataframe
street_df['coordinates'] = list(zip(street_df.Longitude, street_df.Latitude))
street_df['coordinates'] = street_df['coordinates'].apply(Point)
street_gdf = gpd.GeoDataFrame(street_df, geometry='coordinates')
street_gdf = street_gdf.drop(columns=['Latitude', 'Longitude'])
street_gdf.head()

In [ ]:
#street_gdf.plot();

In [ ]:
street_lsoa = street_df.groupby(['LSOA code','Crime type'])['Location'].count().unstack(fill_value=0).reset_index()
street_lsoa.head()

In [ ]:
london_boroughs.head()

In [ ]:
print(street_lsoa['LSOA code'].unique())

In [ ]:
london_boroughs['code'].unique()

In [ ]:
london_boroughs = pd.merge(london_boroughs, street_lsoa, how='outer', sort=True, left_on='code', right_on='LSOA code')
london_boroughs.head()

In [ ]:
london_boroughs.plot();

### Spatial join

In [ ]:
# load data from a url
london_boroughs = gpd.read_file("https://skgrange.github.io/www/data/london_boroughs.json")

In [ ]:
burglaries = street_gdf[street_gdf['Crime type']=='Burglary']
burglaries.head()

In [ ]:
burglaries.crs = 'epsg:4326'

In [ ]:
points1 = gpd.sjoin(boroughs,burglaries) 
points1.head()

In [ ]:
#points2 = pd.pivot_table(points1,index='code',columns='Crime type',aggfunc={'Crime type':'count'})
#points2.columns = points2.columns.droplevel()
#points2 = points2.reset_index()
points2 = points1.groupby(points1['code'])['Crime type'].count()
points2
#fig = plt.figure();
#street_df['Crime type'].groupby(street_df['Month']).count().plot(figsize=(14,6));
#plt.title(force);

# group by outcome
#street_outcome = street_df.groupby(['Month','Last outcome category'])['Location'].count().unstack(fill_value=0)


In [ ]:
london_boroughs = london_boroughs.merge(points2, on='code')
#london_boroughs = london_boroughs.rename(columns={'Crime type': 'Burglary'}, inplace=True)
london_boroughs


## 7. Summary


### Street level crimes

* Crime ID, Month
* Longitude, Latitude (these are an approximation of where the actual crimes occurred, not the exact locations)
* Location, LSOA code, LSOA name
* Crime type
* Last outcome category


### Outcome - case history

* Crime ID, Month
* Longitude, Latitude (these are an approximation of where the actual crimes occurred, not the exact locations)
* Location, LSOA code, LSOA name
* Outcome type


## 8. Open questions and ideas 

### What is in each dataset? Can they be combined? And how, on which level, variables etc?

* All three datasets contain lat-lon

### What other data is needed?

* Check [this list](https://data.police.uk/data/statistical-data/)


### What does fairness mean?

* Different definitions
* Disentangle the judicial process

### What can cause bias?

Available data:
* Age
* Gender
* Race
* Location
    * different regulations?
    * number of schools, shops, pubs etc. - can get this easily from OSM data

### Reconstruct process 

* Use dates + IDs in Street level crimes and Outcome 
* For example, how long does it take to go from arrested to charged?

### Where is the crime rate highest?

* What external data can this be related to? Census data?
* What geospatial data is available to create maps? Probably LSOA shapefiles and lat-lon in 

### Where, when and why have crime rates changed most?


### Are major events easily spotted?

* Number of crimes much lower during start of lockdown
* Changes in laws and regulations
* Change in size of police force
* Are there weekly and seasonal cycles?




